<a href="https://colab.research.google.com/github/bharatnishant/Image-Classifier/blob/master/Cat_vs_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cat vs. Dog Classifier



In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

#Load Dataset

In [0]:
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip Cat_Dog_data.zip

--2019-07-04 06:05:25--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.170.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.170.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580495262 (554M) [application/zip]
Saving to: ‘Cat_Dog_data.zip.1’

Cat_Dog_data.zip.1  100%[===================>] 553.60M  16.7MB/s    in 35s     

2019-07-04 06:06:01 (15.6 MB/s) - ‘Cat_Dog_data.zip.1’ saved [580495262/580495262]

Archive:  Cat_Dog_data.zip
replace Cat_Dog_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#Image Transformation
Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [0]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.Resize(250),
                                       transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(250),
                                      transforms.CenterCrop(224),
                                     transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

#Neural Network

In [0]:
torch.cuda.is_available()

True

In [0]:
## Using a pretrained model to classify the cat and dog images
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:00<00:00, 105444423.05it/s]


In [0]:
model

In [0]:
for param in model.parameters():
  param.requires_grad = False
  
network = nn.Sequential(nn.Linear(2048, 1024),
                     nn.ReLU(),
                     nn.Dropout(p=0.02),
                     nn.Linear(1024, 512),
                     nn.ReLU(),
                     nn.Dropout(p=0.02),
                     nn.Linear(512,2),
                     nn.LogSoftmax(dim =1))

model.fc = network

In [0]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
epochs = 2
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
  for images, labels in trainloader:
    steps += 1
    
    images, labels = images.to('cuda'), labels.to('cuda')
    
    optimizer.zero_grad()
    logp = model(images)
    loss = criterion(logp,labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
    if steps % print_every == 0:
      model.eval()
      test_loss = 0
      accuracy = 0
      
      for images, labels in testloader:
        images, labels = images.to('cuda'), labels.to('cuda')
        logp = model(images)
        loss = criterion(logp,labels)
        test_loss+=loss.item()
        
        ps = torch.exp(logp)
        top_ps, top_class = ps.topk(1, dim=1)
        equal = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equal.type(torch.FloatTensor)).item()
        
      print(f"{epoch+1}/{epochs}")
      print(f"Train Loss: {running_loss/print_every}")
      print(f"Test Loss: {test_loss/len(testloader)}")
      print(f"Accuracy: {accuracy/len(testloader)}")
      
      running_loss = 0
      model.train()

1/2
Train Loss: 0.0754825484007597
Test Loss: 0.15566618852317332
Accuracy: 0.93203125
1/2
Train Loss: 0.07162664383649826
Test Loss: 0.18881676169112324
Accuracy: 0.91875
1/2
Train Loss: 0.15416047722101212
Test Loss: 0.20267999768257142
Accuracy: 0.921484375
1/2
Train Loss: 0.055309022217988967
Test Loss: 0.20461415629833937
Accuracy: 0.925390625
1/2
Train Loss: 0.057552133500576017
Test Loss: 0.1992176384665072
Accuracy: 0.909375
1/2
Train Loss: 0.024494681879878043
Test Loss: 0.1942946221679449
Accuracy: 0.91171875
1/2
Train Loss: 0.05145808551460505
Test Loss: 0.15278966715559364
Accuracy: 0.930859375
1/2
Train Loss: 0.04762485343962908
Test Loss: 0.28116775618400425
Accuracy: 0.88046875
1/2
Train Loss: 0.041114870458841324
Test Loss: 0.21296418071724474
Accuracy: 0.90546875
1/2
Train Loss: 0.03455227054655552
Test Loss: 0.14729456026107074
Accuracy: 0.937890625
1/2
Train Loss: 0.04265879690647125
Test Loss: 0.16343209529295563
Accuracy: 0.928515625
1/2
Train Loss: 0.0388913152739